<a href="https://colab.research.google.com/github/andrey-varna/OSR_passport/blob/main/Stazhirovka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''скачиваю clone YOLOv5 ''' 
!git clone https://github.com/ultralytics/yolov5                                # Скачиваю clone с репрозитория
%cd yolov5
%pip install -qr requirements.txt                                               # Устанавливаю зависимости
%pip install -q roboflow                                                        # Устанавливаю roboflow 
''' устанавливаю дополнительные модули '''
import torch                                                                    # Импортирую библиотеку torch
import os                                                                       # Модуль работы с файловой системой
from IPython.display import Image, clear_output                                 # Модуль для вывода изображений

Cloning into 'yolov5'...
remote: Enumerating objects: 15305, done.
remote: Total 15305 (delta 0), reused 0 (delta 0), pack-reused 15305
Receiving objects: 100% (15305/15305), 14.13 MiB | 24.19 MiB/s, done.
Resolving deltas: 100% (10501/10501), done.
/content/yolov5/yolov5


In [ ]:
from google.colab import drive                                                  # Подключаю гугл диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from PIL import Image, ImageFont
from skimage import io

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")
clear_output()

In [ ]:
model_round = torch.hub.load('/content/yolov5/', 'custom', path='/content/drive/MyDrive/AI/Proekts/pasport/rotation.pt', source='local')

def rotate_image(mat, angle):
    height, width = mat.shape[:2]
    image_center = (width / 2, height / 2)
    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)
    abs_cos = abs(rotation_mat[0, 0])
    abs_sin = abs(rotation_mat[0, 1])
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)
    rotation_mat[0, 2] += bound_w / 2 - image_center[0]
    rotation_mat[1, 2] += bound_h / 2 - image_center[1]
    return cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))

def get_angle_rotation(centre, point, target_angle):
    new_point =(point[0] - centre[0], point[1] - centre[1])
    a,b = new_point[0], new_point[1]
    res = math.atan2(b,a)
    if (res < 0) :
          res += 2 * math.pi
    return (math.degrees(res)+target_angle) % 360

def result(img):
    return model_round(img)

def get_image_after_rotation(img):
        #запихнули в первую сетку для поворота она что-то вернула далее на основании результатов как-то определили углы на которые надо поврнуть.
    results = result(img)
    pd = results.pandas().xyxy[0]
    pd = pd.assign(centre_x = pd.xmin + (pd.xmax-pd.xmin)/2)
    pd = pd.assign(centre_y = pd.ymin + (pd.ymax-pd.ymin)/2)
    tmp = pd.loc[pd['name']=='niz']
    N, V = None, None
    for index, row in tmp.iterrows():
        N = (row['centre_x'], row['centre_y'])
        break
    tmp = pd.loc[pd['name']=='verh']
    for index, row in tmp.iterrows():
        V = (row['centre_x'], row['centre_y'])
        break
    if N == None or V == None:
        return img
        #здесь определение угла и поворот картинки
    angle = get_angle_rotation(N, V, 90)
    img = rotate_image(img, angle)
    return img

def crop_img(img):
    results = result(img)
    pd = results.pandas().xyxy[0]
    x1 =int(pd.xmin.min())
    x2 = int(pd.xmax.max())
    y1 = int(pd.ymin.min())
    y2 = int(pd.ymax.max())
    img = img[y1:y2,x1:x2]
    return img

def get_crop(file):
    image = cv2.imread(file)
    '''Повышаю резкость изображения '''
    sharp_filter = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    # параметр ddepth отвечает за «глубину» картинки
    # ddepth=-1 означает, что глубина получившейся картинки будет как у исходной
    sharpen_img = cv2.filter2D(image, ddepth=-1, kernel=sharp_filter)
    '''перевожу в оттенки серого'''
    image = cv2.cvtColor(sharpen_img, cv2.COLOR_BGR2GRAY)
    '''нормализовываю яркость изображения'''
    normalized = cv2.equalizeHist(image)
    ''' применяю гауссово размытие для сглаживания засветки '''
    blurred = cv2.GaussianBlur(normalized, (0, 0), 5)
    '''вычисляю маску области с засветкой '''
    mask = cv2.threshold(blurred, 240, 255, cv2.THRESH_BINARY)[1]
    '''удаляю засветку из оригинального изображения '''
    removed = cv2.inpaint(image, mask, 5, cv2.INPAINT_TELEA)
    '''поворот в нормальный вид с помощью первой yolo '''
    image = get_image_after_rotation(removed)
    image = get_image_after_rotation(image)
            #обрезка
    image = crop_img(image)
    return image

#get_crop('/content/drive/MyDrive/AI/Diplom/Photo/фото_363.png') 

YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)



[Errno 2] No such file or directory: '/usr/local/lib/python3.9/dist-packages/pyparsing-3.0.9.dist-info/METADATA'


Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [ ]:
# Подготавливаю базу для сегментации, то счем будет работать вторая нейронка после подготовки фото
# распознанные фото остаются в базе, не распознанные - удаляю (приходится делать вручную)
path = '/content/drive/MyDrive/AI/Proekts/pasport/'

#files = os.listdir(path + 'foto/')                                             # Получаем список файлов
files = os.listdir(path+'images/' ) 
print('осталось обработать -', len(files))                                      # вывожу количество не обработыннх файлов
n = 1
for f in files:
    filename = f.split('.')[0]
    print(filename)                                                             # вывожу имена распознанных и не распознанного, на нем колаб вылетает
    image = get_crop(path+'images/'+filename+'.jpg')                            # запускаю первую сетку для переворота и перевода в серые цвета
    image1 = Image.fromarray(image)
    image1.save(path+'/foto_val/'+'foto_'+f'{n}''.png', quality=100)
    n+=1
    #os.remove(path+'images/'+f)
    image1.close()

осталось обработать - 280
001
2-3
352


ValueError: ignored

In [ ]:
'''Подготовка фото к обработке сетью'''
# фотки в базе были с длинными именами содержали в имени еще по расширению с точкой, надо было избавиться от лишнего
import os
folder_path = '/content/drive/MyDrive/AI/Proekts/pasport/rotation_main/valid'
extension = '.jpg'
for filename in os.listdir(folder_path):

    ''' проверка наличия точки в имени'''
    if '.' in filename:

        ''' получаю последние 5 знаков перед точкой '''
        name, extension = os.path.splitext(filename)
        #new_filename = name[-:] + extension
        new_filename = name[-5:] + '.jpg'

        ''' перенаименовываю файл'''
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))

## Обучаю вторую сетку на разпознание областей

Передаю при запуске ряд аргументов:
- **img:** для определения размера входного изображения
- **batch:** для определения размера обучаешейся партии 
- **epochs:** задаю количество эпох обучения. 
- **data:** местоположение набора данных сохраняется в `dataset.location`, преписываю под свою задачу конфигурационный файл coco128.yaml, который потом запускается(переписанный файл сохраняю под именем coco128_1.yaml)
- **weights:** указываю путь к весам, с которых начинаю перенос обучения, здесь беру веса с дипломной работы.
- **cache:** указываю кешировать изображения для более быстрого обучения

In [ ]:
!python train.py --img 1024 --batch 12 --epochs 100 --data /content/drive/MyDrive/AI/Proekts/pasport/rotation_main/coco128_1.yml --weights /content/drive/MyDrive/AI/Proekts/pasport/rotation_main/rotation.pt --cache

train: weights=/content/drive/MyDrive/AI/Proekts/pasport/rotation_main/rotation.pt, cfg=, data=/content/drive/MyDrive/AI/Proekts/pasport/rotation_main/coco128_1.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=12, imgsz=1024, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, 

In [ ]:
#model_round = torch.hub.load('/content/yolov5/', 'custom', path='/content/drive/MyDrive/AI/Proekts/pasport/rotation.pt', source='/content/drive/MyDrive/AI/Proekts/pasport/images/rt_65.jpg')
!python detect.py --img 640 --conf 0.2 --source data/images --weights /content/drive/MyDrive/AI/Proekts/pasport/rotation.pt --img 640 --conf 0.1 --source /content/drive/MyDrive/AI/Proekts/pasport/rotation_main/test_rot/352.jpg

detect: weights=['/content/drive/MyDrive/AI/Proekts/pasport/rotation.pt'], source=/content/drive/MyDrive/AI/Proekts/pasport/rotation_main/test_rot/352.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/1 /content/drive/MyDrive/AI/Proekts/pasport/rotation_main/test_rot/352.jpg: 480x640 1 pasport, 23.2ms
Speed: 0.6ms pre-process, 23.2ms inference, 1.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp2


In [ ]:
!python train.py --img 1024 --batch 12 --epochs 100 --data /content/drive/MyDrive/AI/Proekts/pasport/coco128_1.yaml --weights /content/drive/MyDrive/AI/Diplom/best.pt --cache